## Scrape Data from Widipedia

This notebook is created to scrape the Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [2]:
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

We use the 'request' and 'BeautifulSoup' libraries to get the 'lxml' file and then find the all table tags for subsequently generating dataframe.

In [27]:
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(website_url).text

soup = BeautifulSoup(results, 'lxml')
my_table = soup.find('table', {'class': "wikitable sortable"})
table = my_table.findAll('td')

Build a loop to extract all data and store as dictionary.

In [51]:
col_list = ['PostCode', 'Borough', 'Neighbourhood']
data_dict = {}
for ind, key in enumerate(col_list):
    i = ind
    value = []
    while i <= len(table)-1:
        text = table[i].text
        value.append(text)
        i += 3
    data_dict[key] = value

print('Store data in dictionary!')

0 PostCode
1 Borough
2 Neighbourhood


Build dataframe and clean the dataset.

In [92]:
# Build and clean dataframe
data_df = pd.DataFrame(data_dict)
data_df = data_df[['PostCode', 'Borough', 'Neighbourhood']]
data_df['Neighbourhood'] = data_df['Neighbourhood'].apply(lambda x: x.split("\n")[0])

# Drop the cells with Not assigned Borough
table_df = table_df[table_df.Borough != 'Not assigned']

# A cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
table_df['Neighbourhood'][table_df.Neighbourhood == 'Not assigned'] = table_df[table_df.Neighbourhood == 'Not assigned']['Borough']

# Two rows are combined into one row with same PostCode and Borough.
table_df = table_df.groupby(['PostCode', 'Borough']).agg({'Neighbourhood': lambda x: ' , '.join(x)}).reset_index()

In [97]:
# Have a look the data
table_df.head()

,PostCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [100]:
#Print the row number of the cleaned data
print('Row number of cleaned data:', table_df.shape[0])

Row number of cleaned table: 103
